<a href="https://colab.research.google.com/github/Krishnaa548/KNN_Model/blob/main/Book_recommendation_qwen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Cell 1: Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [ ]:
books_df = pd.read_csv('/content/BX-Books.csv', sep=';', encoding='ISO-8859-1', on_bad_lines='skip') # Use on_bad_lines='skip' to skip bad lines
ratings_df = pd.read_csv('/content/BX-Book-Ratings.csv', sep=';', encoding='ISO-8859-1', on_bad_lines='skip') # Use on_bad_lines='skip' to skip bad lines

<ipython-input-25-754f0186dedd>:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books_df = pd.read_csv('/content/BX-Books.csv', sep=';', encoding='ISO-8859-1', on_bad_lines='skip') # Use on_bad_lines='skip' to skip bad lines


In [ ]:
# Cell 4: Data filtering
# Filter users with <200 ratings
user_counts = ratings_df['User-ID'].value_counts()
filtered_users = user_counts[user_counts >= 200].index
filtered_ratings = ratings_df[ratings_df['User-ID'].isin(filtered_users)]

In [ ]:
# Filter books with <100 ratings among filtered users
book_counts = filtered_ratings['ISBN'].value_counts()
filtered_books = book_counts[book_counts >= 100].index
final_ratings = filtered_ratings[filtered_ratings['ISBN'].isin(filtered_books)]

In [ ]:
# Cell 5: Merge data and prepare for matrix
merged_df = pd.merge(final_ratings, books_df, on='ISBN')
# Calculate mean only on 'Book-Rating' column
merged_df = merged_df.groupby(['User-ID', 'ISBN', 'Book-Title'])['Book-Rating'].mean().reset_index()

In [ ]:
# Cell 6: Create book-user interaction matrix
book_user_matrix = merged_df.pivot(
    index='ISBN',
    columns='User-ID',
    values='Book-Rating'
).fillna(0)

In [ ]:
# Cell 7: Train the KNN model
model_knn = NearestNeighbors(metric='cosine', algorithm='brute')
model_knn.fit(book_user_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [ ]:
# Cell 8: Create title-to-ISBN mapping
title_to_isbn = merged_df[['Book-Title', 'ISBN']].drop_duplicates().set_index('Book-Title')['ISBN'].to_dict()
isbn_to_title = books_df.set_index('ISBN')['Book-Title'].to_dict()

In [ ]:
def get_recommends(title):
    isbn = title_to_isbn.get(title)
    if not isbn:
        return []

    # Get book vector
    book_vector = book_user_matrix.loc[[isbn]]  # Keep as DataFrame

    # Find neighbors
    distances, indices = model_knn.kneighbors(book_vector, n_neighbors=6)

    # Convert row indices to ISBNs
    neighbor_indices = book_user_matrix.index[indices.squeeze()].tolist()

    # Exclude first entry (the book itself)
    neighbor_isbns = neighbor_indices[1:6]

    recommendations = []
    for i in range(5):
        neighbor_isbn = neighbor_isbns[i]
        distance = distances[0][i+1]
        neighbor_title = isbn_to_title[neighbor_isbn]
        recommendations.append([neighbor_title, distance])

    return [title, recommendations]


In [ ]:
# Cell 10: Test the function (this is the test cell provided)
print(get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))"))

[]


Other way BOOk recommendation

In [ ]:
# Libraries for data preparation & visualization
import numpy as np
import pandas as pd
import plotly.offline as py
import plotly.graph_objs as go
import plotly.io as pio
pio.renderers.default = "png"

# Ignore printing warnings for general readability
import warnings
warnings.filterwarnings("ignore")

In [ ]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 2.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp311-cp311-linux_x86_64.whl size=2505181 sha256=35410f8333db74bca9b248967b929b9031c8c54bd5f86a8527514e5aa3fbcaf6
  Stored in directory: /root/.cache/pip/wheels/2a/8f/6e/7e2899163e2d85d8266daab4aa1cdabec7a6c56f83c015b5af
Successfully built scikit-surprise


In [ ]:
# pip install scikit-surprise # This line was a comment and did not execute, causing the error
# Importing libraries for model building & evaluation
from sklearn.model_selection import train_test_split
from surprise import Reader, Dataset
from surprise.model_selection import train_test_split, cross_validate, GridSearchCV
from surprise import KNNBasic, KNNWithMeans, KNNWithZScore, KNNBaseline, SVD
from surprise import accuracy

In [ ]:
# Loading the dataset
def loaddata(filename):
    df = pd.read_csv(f'{filename}.csv',sep=';',error_bad_lines=False,warn_bad_lines=False,encoding='latin-1')
    return df

book   = loaddata("BX-Books")
user   = loaddata("BX-Users")
rating = loaddata("BX-Book-Ratings")

TypeError: read_csv() got an unexpected keyword argument 'error_bad_lines'

# New Section